# Summing two binomial distributions

In [1]:
from scipy.stats import binom

## Background question

Given a job vacancy with multiple candidates requested, what is the chance `n_requested` gets fulfilled?

What we know:
* Candidates that have an interview scheduled (`n_interviews`) have a certain chance of getting hired (`p_interviews`)
* Candidates that have applied for the job (`n_applications`) have a certain chance of getting hired (`p_applications`)

The chances of getting hired (true or false) for both interview and application candidates comes from historical data and can best be modelled as a binomial distribution with parameters `n` and `p`.

### The problem

If `n_interviews < n_requested` and `n_applications < n_requested` there is no chance the vacancy is going to be completely fulfilled with candidates from one pool only. This means we have to look into combinations of both interview and applications candidates to fulfill the vacancy.

Consider the following example case:

In [2]:
n_requested = 3
n_applications = 2
p_applications = 0.1
n_interviews = 2
p_interviews = 0.3

## Binomial distribution

First, let's understand the binomial distribution and how we can model the chance of a request being fulfilled by just a single type of candidates first. When we understand this, we can increase the example from one to two binomial distributions.

A binomial distribution can be created using `scipy.stats.binom`:

In [3]:
binom_app = binom(n_applications, p_applications)

The **probability mass function (PMF)** is then a function that gives the probability that a discrete random variable is exactly equal to some value. 

So printing the odds that exactly `k` people get hired:

In [4]:
for k in range (n_requested + 1):
    print(k, binom_app.pmf(k))

0 0.81
1 0.18
2 0.01
3 0.0


In other words: there's 18% chance one applicant gets hired. And 0% that 3 applicants get hired (this is also not possible as only 2 candidates are applying).

The **cumulative distribution function (CDF)** will print the probability that X will take a value less than or equal to some number. This considers all possible combinations that are possible. 

More usefull in our case is to look at it's complement, which gives us the probability that `more than k` applicants get hired:

In [5]:
for k in range (n_requested + 1):
    print(k, 1 - binom_app.cdf(k))

0 0.19
1 0.01
2 0.0
3 0.0


So there's a 19% chance 1 or 2 applicants get hired and a 1% chance 2 applicants get hired.

We can make this a generic function, that calculates the chance of a specific `n_request` being fulfilled by some binomial distributions with parameters `n_population` and `p_hired`:

In [6]:
def p_request_fulfilled(n_requested, n_population, p_hired):
    "Calculate chance of request being fulfilled depending on population size and the chance of getting hired"
    binom_dist = binom(n_population, p_hired)
    return 1 - binom_dist.cdf(n_requested - 1)

print('Chance of request being fulfilled:', 
      p_request_fulfilled(1, 2, 0.1))

Chance of request being fulfilled: 0.19


In [7]:
print('Chance of request being fulfilled:', 
      p_request_fulfilled(2, 2, 0.1))

Chance of request being fulfilled: 0.01


In [8]:
print('Chance of request being fulfilled:', 
      p_request_fulfilled(1, 1, 0.1))

Chance of request being fulfilled: 0.1


These outcomes make sense.

## Summing two binomial distributions?

As state earlier, the problem is that both distributions indicate that there is 0 chance of fulfilling the `n_requested` of 3 on their own. We thus have to look at their distributions together and investigate the area of overlap.

### Manual calculation for ground truth

There are 3 ways in which we can fullfill the demand of `n_requested = 3`:

* 2 succesfull applicants + 1 succesfull interviewiee
* 1 succesfull applicant  + 2 succesfull interviewiees
* 2 succesfull applicants + 2 succesfull interviewiees

Let's calculate what the output chance should be using **PMF** to calculate the above exact possibilities:

In [9]:
binom_app = binom(n_applications, p_applications)
binom_int = binom(n_interviews, p_interviews)

# Calculate true chance of 3 requests to be fulfilled from two binomial distributions
truth = binom_app.pmf(2) * binom_int.pmf(1) + \
        binom_app.pmf(1) * binom_int.pmf(2) + \
        binom_app.pmf(2) * binom_int.pmf(2)

print('True chance:', truth)

True chance: 0.0213


So this is what our final function should output for this specific case.

### Theory

What is the distribution of the variable 𝑋 given: 

𝑋=𝑌+𝑍,

where 𝑌∼ Binomial(𝑛, 𝑃𝑌) and 𝑍∼ Binomial(𝑛, 𝑃𝑍)?

According to [wikipedia](https://en.wikipedia.org/wiki/Binomial_sum_variance_inequality):

> the sum of independent binomial random variables is itself a binomial random variable if all the component variables share the same success probability. If success probabilities differ, the probability distribution of the sum is not binomial.

As in our case 𝑃𝑌 != 𝑃𝑍, this means we know that 𝑌 will **not** be binomial itself.. meaning theory doesn't help us.


### Practice

An option would be to use a more "brute force" kind of strategy, wherein we consider both distributions and use the probability mass function to sum all the probabilities where `i_applications + j_interviews > n_requested`:

In [10]:
def p_request_fulfilled_two_binomials(n_requested, n1, n2, p1, p2):
    """Calculated the chance of a requested being fulfilled by considering all 
       combinations from two binomial distributions"""
    all_probs = 0
    binom1 = binom(n1, p1)
    binom2 = binom(n2, p2)
    
    # loop over the number of candidates in application
    for i in range(n1 + 1):
        # loop over the number of candidates scheduled for interview
        for j in range(n2 + 1):
            # consider if the cases fulfills the number of candidates requested 
            if i + j >= n_requested:
                # calculate the chance of that scenario happening and add it to the total
                all_probs += binom1.pmf(i) * binom2.pmf(j)
    return all_probs
            
print('Chance of request being fulfilled:',
      p_request_fulfilled_two_binomials(n_requested, n_applications, n_interviews, p_applications, p_interviews))

Chance of request being fulfilled: 0.0213


Looks good, as that matches with the `truth` that we expected. 

Let's try with another scenario:

In [11]:
n_requested = 10
n_applications = 8
p_applications = 0.5
n_interviews = 8
p_interviews = 0.5

print('Chance of request being fulfilled:',
      p_request_fulfilled_two_binomials(n_requested, n_applications, n_interviews, p_applications, p_interviews))

Chance of request being fulfilled: 0.227249145508


Still makes sense. Now what if the request should be easily fulfilled?

In [12]:
n_requested = 1
n_applications = 8
p_applications = 0.1
n_interviews = 8
p_interviews = 0.3

print('Chance of request being fulfilled:',
      p_request_fulfilled_two_binomials(n_requested, n_applications, n_interviews, p_applications, p_interviews))

Chance of request being fulfilled: 0.975184421973


Great!

## Next steps

The above calculations don't give you any uncertainty estimate. 

Possibly you could get this by using `pymc3` sampling methods.

Done.